Stock price tracer

In [1]:
#!pip install yfinance
#!pip install nltk
#import nltk
#nltk.download()
#!pip install mysql-connector-python
#!pip install psycopg2
#!pip install schedule

Companies interested

In [2]:
company_names = {
    "Apple": "AAPL",
    "Google": "GOOGL",
    "NVDIA": "NVDA",
    "Tesla": "TSLA",
    "Amazon": "AMZN",
    "Procter & Gamble": "PG",
    "Microsoft" : "MSFT",
    "AMD" : "AMD", 
    "SPDR_S&P500" : "SPY",
    "AT&T" : "T", 
    "Palantir Technologies" : "PLTR",
    "Meta Platforms" : "META",
    "Johnson&Johnson" : "JNJ",
    "The Coca-Cola Company" : "KO"
    
}


In [3]:
import yfinance as yf

def get_stock_price(stock):
    current_datetime = datetime.now()
    formatted_datetime = current_datetime.strftime("%Y-%m-%d %H:%M:%S")
    today_datetime = current_datetime.strftime("%Y-%m-%d")
    ticker = stock
    stock = yf.Ticker(ticker)
    history = stock.history(period='1d', interval='1d', actions=False)
    history = history[~history.index.isin([today_datetime])]  # Exclude the problematic date
    price = history['Close'].iloc[-1]  # Get the latest closing price
    return price


from datetime import datetime, timedelta
import pytz

def get_stock_diff(stock):
    ticker = stock
    stock = yf.Ticker(ticker)
    history = stock.history(period='1d', interval='1m', actions=False)
    current_time = datetime.now(pytz.timezone('America/New_York'))
    one_hour_before = current_time - timedelta(hours=1)
    one_hour_before = one_hour_before.replace(tzinfo=pytz.timezone('America/New_York'))
    history_one_hour_before = history.loc[history.index >= one_hour_before]
    price_one_hour_before = history_one_hour_before['Close'].iloc[0]
    return price_one_hour_before



Relate Keyword 

In [4]:
import requests
from bs4 import BeautifulSoup
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
# Perform a search on Google News



def analyze_webpage(keyword):
    #url1 = f"https://www.google.com/search?q=&tbm=nws"
    #url =  f"https://www.google.com/finance/quote/{keyword}:NASDAQ?hl=en"
    url = f"https://www.google.com/search?q={keyword}+stock&tbs=sbd:1,qdr:h&tbm=nws&sxsrf=AB5stBiVFYdoWM80R6YGsAMPttTAQSWaYQ:1689030176807&source=lnt&sa=X&ved=2ahUKEwjN_8_dn4WAAxVvkWoFHV9KD0cQpwV6BAgBEAc&biw=1042&bih=774&dpr=1.25"
    # Fetch the HTML content of the webpage
    response = requests.get(url)
    response.raise_for_status()

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.text, "html.parser")

    # Extract the text content from the webpage
    text = soup.get_text()

    # Tokenize the text into individual words
    tokens = word_tokenize(text)

    # Remove stopwords (common words with little meaning)
    stop_words = set(stopwords.words("english"))
    s= ['searchgoogle×please','google','apple', 'tesla', 'nvdia',  'click', 'allnewsshoppingimages', 'dateshowing', 'results', 'nvidiasearch', 'instead', 'nvdianvidia','redirected', "hours", "agoPopular",'within', 'allnewsimagesshopping','seconds', 'maps', 'videos', 'books', 'search', 'tools', 'recentrecentpast', 'hourpast', '24', 'hourspast', 'weekpast', 'monthpast', 'yeararchivessorted', 'relevancesorted', 'relevancesorted', 'datenews', 'vision', 'pro', 'applebuying', 'apple', 'vision', 'pro', 'headset', 'could', 'require', 'appointment','allnewsvideosimages', 'shopping', 'past', 'hoursrecentpast', 'datesorted', 'nvidia', 'newssearch', 'nvdia', 'newsnvidia']
    for _ in s:
        stop_words.add(_)
    filtered_tokens = [word.lower() for word in tokens if word.lower() not in stop_words and re.match(r'\b\w+\b', word)]

    # Perform sentiment analysis on the text
    sia = SentimentIntensityAnalyzer()
    sentiment_scores = sia.polarity_scores(' '.join(filtered_tokens))
    sentiment = sentiment_scores['compound']

    # Print the results
    return sentiment


Updating stock price data on Database

In [5]:
import psycopg2
from datetime import datetime


def upload_data():
    for company_name in company_names.keys():
        current_datetime = datetime.now()
        formatted_datetime = current_datetime.strftime("%Y-%m-%d %H:%M:%S")
        today_datetime = current_datetime.strftime("%Y-%m-%d")
    
        price = get_stock_price(company_names[company_name])
        sentiment = analyze_webpage(company_names[company_name])
        connection = psycopg2.connect(
            host="localhost",
            port="5432",
            database="postgres",
            user="postgres",
            password="0811"
        )   

        cursor = connection.cursor()

        query = "INSERT INTO stock_table (date, company, price, sentiment) VALUES (%s, %s, %s, %s)"

        # Get the current date and time
        data = [(formatted_datetime, company_name, price, sentiment)]
        print(data)
        cursor.executemany(query, data)
 
        connection.commit()

        cursor.close()
        connection.close()



Schedule function

In [6]:
import schedule
import time

def job():
    upload_data()
    print("Running my_function on weekdays...")

# Schedule the job to run every hour from 9 AM to 4 PM on weekdays
schedule.every().day.at("09:00").do(job)  # Schedule the job to start at 9 AM

# Keep the script running
while True:
    current_time = time.strftime("%H:%M")
    if current_time >= "09:00" and current_time < "16:00":
        job()
    time.sleep(1200)  # Sleep for 1 hour


[('2023-07-11 10:41:30', 'Apple', 187.4949951171875, 0.8658)]
[('2023-07-11 10:41:31', 'Google', 117.08499908447266, 0.8442)]
[('2023-07-11 10:41:32', 'NVDIA', 422.3900146484375, 0.5574)]
[('2023-07-11 10:41:33', 'Tesla', 267.5400085449219, 0.9539)]
[('2023-07-11 10:41:33', 'Amazon', 128.20989990234375, 0.8658)]
[('2023-07-11 10:41:34', 'Procter & Gamble', 147.89999389648438, 0.8625)]
[('2023-07-11 10:41:35', 'Microsoft', 329.7200012207031, -0.9652)]
[('2023-07-11 10:41:36', 'AMD', 109.9000015258789, 0.7506)]
[('2023-07-11 10:41:37', 'SPDR_S&P500', 440.55999755859375, 0.0)]
[('2023-07-11 10:41:37', 'AT&T', 15.204999923706055, 0.6705)]
[('2023-07-11 10:41:38', 'Palantir Technologies', 16.5049991607666, 0.0)]
[('2023-07-11 10:41:39', 'Meta Platforms', 297.2300109863281, 0.2023)]
[('2023-07-11 10:41:39', 'Johnson&Johnson', 158.82000732421875, 0.2732)]
[('2023-07-11 10:41:40', 'The Coca-Cola Company', 59.28499984741211, 0.7717)]
Running my_function on weekdays...
[('2023-07-11 11:01:41', '